In [1]:
data = []
with open("Day08.txt") as file:
    for line in file:
        a, b = line.strip().split(' | ')
        a = tuple(frozenset(item) for item in a.split())
        b = tuple(frozenset(item) for item in b.split())
        data.append((a, b))

In [2]:
%%time
from collections import Counter

counter = Counter()
for left, right in data:
    counter.update(len(item) for item in right)

counter[2] + counter[3] + counter[4] + counter[7]

CPU times: user 355 µs, sys: 0 ns, total: 355 µs
Wall time: 365 µs


274

In [3]:
def identify(patterns):
    # Segments
    # ┌ a ┐
    # b   c
    # ├ d ┤
    # e   f
    # └ g ┘
    values = {}
    # Unique patterns (1, 4, 7, 8)
    for value, count in ((1, 2), (4, 4), (7, 3), (8, 7)):
        values[value] = next(p for p in patterns if len(p) == count)
    # 5-segments (2, 3, 5)
    # Bottom-left corner segments
    segments_5 = tuple(p for p in patterns if len(p) == 5)
    segment_a = values[7] - values[1]
    segments_e_g = values[8] - values[4] - segment_a
    segment_g = segments_e_g - frozenset.union(*(segments_e_g - s for s in segments_5))
    segment_e = segments_e_g - segment_g
    # Deducted patterns from bottom-left corner (2, 3, 5)
    values[2] = next(p for p in segments_5 if p & segment_e)
    values[3] = next(p for p in segments_5 if p & values[1] == values[1])
    values[5] = next(p for p in segments_5 if p not in values.values())
    # 6-segments (0, 6, 9)
    segments_6 = tuple(p for p in patterns if len(p) == 6)
    values[9] = next(p for p in segments_6 if not p & segment_e)
    values[0] = next(p for p in segments_6 if p != values[9] and p & values[1] == values[1])
    values[6] = next(p for p in segments_6 if p not in values.values())
    # Return result
    return values

In [4]:
def translate(sequence, values):
    translator = {value: key for key, value in values.items()}
    return sum(translator[value] * (10 ** exp) for exp, value in enumerate(sequence[::-1]))

In [5]:
%%time
sum(translate(right, identify(left)) for left, right in data)

CPU times: user 3.44 ms, sys: 52 µs, total: 3.49 ms
Wall time: 3.46 ms


1012089